In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

%matplotlib inline

In [2]:
may_trip = pd.read_csv('../data/may_trip.csv')

In [3]:
may_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224969 entries, 0 to 224968
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pubTimeStamp    224969 non-null  object 
 1   companyName     224969 non-null  object 
 2   tripRecordNum   224969 non-null  object 
 3   sumdID          224969 non-null  object 
 4   tripDuration    224969 non-null  float64
 5   tripDistance    224969 non-null  float64
 6   startDate       224969 non-null  object 
 7   startTime       224969 non-null  object 
 8   endDate         224969 non-null  object 
 9   endTime         224969 non-null  object 
 10  startLatitude   224969 non-null  float64
 11  startLongitude  224969 non-null  float64
 12  endLatitude     224969 non-null  float64
 13  endLongitude    224969 non-null  float64
 14  tripRoute       224969 non-null  object 
 15  create_dt       224969 non-null  object 
dtypes: float64(6), object(10)
memory usage: 27.5+ MB


In [4]:
may_trip['start_dt'] = pd.to_datetime(pd.to_datetime(may_trip['startDate']).astype(str) + ' ' + may_trip['startTime'])
may_trip['end_dt'] = pd.to_datetime(pd.to_datetime(may_trip['endDate']).astype(str) + ' ' + may_trip['endTime'])

In [5]:
may_trip.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt
0,2019-05-01 00:00:55.423000,Bird,BRD2134,Powered9EAJL,3.0000,958.00528,2019-05-01 00:00:00,00:00:20.460000,2019-05-01 00:00:00,00:02:52.346666,36.15710,-86.80360,36.15660,-86.80670,"[[36.157235,-86.803612],[36.157235,-86.80362],...",2019-05-02 05:30:23.780000,2019-05-01 00:00:20.460000,2019-05-01 00:02:52.346666
1,2019-05-01 00:03:33.147000,Lyft,LFT5,Powered296631,1.7156,1371.39112,2019-05-01 00:00:00,00:01:50.090000,2019-05-01 00:00:00,00:03:33.026666,36.15797,-86.77896,36.16054,-86.77689,"[[36.15797,-86.77896],[36.15795,-86.77873],[36...",2019-05-02 07:20:32.757000,2019-05-01 00:01:50.090000,2019-05-01 00:03:33.026666
2,2019-05-01 00:05:55.570000,Bird,BRD2168,Powered7S2UU,3.0000,2296.58800,2019-05-01 00:00:00,00:03:47.363333,2019-05-01 00:00:00,00:07:13.596666,36.15470,-86.78180,36.15650,-86.78680,"[[36.155068,-86.782124],[36.156597,-86.78675]]",2019-05-02 05:30:24.530000,2019-05-01 00:03:47.363333,2019-05-01 00:07:13.596666
3,2019-05-01 00:05:55.570000,Bird,BRD2166,PoweredZIIVX,3.0000,1200.78744,2019-05-01 00:00:00,00:04:21.386666,2019-05-01 00:00:00,00:06:59.176666,36.14940,-86.77950,36.15310,-86.77960,"[[36.149741,-86.779344],[36.149741,-86.779327]...",2019-05-02 05:30:24.237000,2019-05-01 00:04:21.386666,2019-05-01 00:06:59.176666
4,2019-05-01 00:05:55.570000,Bird,BRD2165,PoweredJ7MB3,2.0000,351.04988,2019-05-01 00:00:00,00:04:27.796666,2019-05-01 00:00:00,00:06:23.150000,36.17780,-86.78660,36.17740,-86.78760,"[[36.177699,-86.786477],[36.177711,-86.786469]...",2019-05-02 05:30:24.207000,2019-05-01 00:04:27.796666,2019-05-01 00:06:23.150000


In [6]:
#may_trip['time_diff'] = (may_trip.endTime - may_trip.startTime )
may_trip['time_diff'] = (may_trip.end_dt - may_trip.start_dt )

In [7]:
may_trip.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
0,2019-05-01 00:00:55.423000,Bird,BRD2134,Powered9EAJL,3.0000,958.00528,2019-05-01 00:00:00,00:00:20.460000,2019-05-01 00:00:00,00:02:52.346666,36.15710,-86.80360,36.15660,-86.80670,"[[36.157235,-86.803612],[36.157235,-86.80362],...",2019-05-02 05:30:23.780000,2019-05-01 00:00:20.460000,2019-05-01 00:02:52.346666,0 days 00:02:31.886666
1,2019-05-01 00:03:33.147000,Lyft,LFT5,Powered296631,1.7156,1371.39112,2019-05-01 00:00:00,00:01:50.090000,2019-05-01 00:00:00,00:03:33.026666,36.15797,-86.77896,36.16054,-86.77689,"[[36.15797,-86.77896],[36.15795,-86.77873],[36...",2019-05-02 07:20:32.757000,2019-05-01 00:01:50.090000,2019-05-01 00:03:33.026666,0 days 00:01:42.936666
2,2019-05-01 00:05:55.570000,Bird,BRD2168,Powered7S2UU,3.0000,2296.58800,2019-05-01 00:00:00,00:03:47.363333,2019-05-01 00:00:00,00:07:13.596666,36.15470,-86.78180,36.15650,-86.78680,"[[36.155068,-86.782124],[36.156597,-86.78675]]",2019-05-02 05:30:24.530000,2019-05-01 00:03:47.363333,2019-05-01 00:07:13.596666,0 days 00:03:26.233333
3,2019-05-01 00:05:55.570000,Bird,BRD2166,PoweredZIIVX,3.0000,1200.78744,2019-05-01 00:00:00,00:04:21.386666,2019-05-01 00:00:00,00:06:59.176666,36.14940,-86.77950,36.15310,-86.77960,"[[36.149741,-86.779344],[36.149741,-86.779327]...",2019-05-02 05:30:24.237000,2019-05-01 00:04:21.386666,2019-05-01 00:06:59.176666,0 days 00:02:37.790000
4,2019-05-01 00:05:55.570000,Bird,BRD2165,PoweredJ7MB3,2.0000,351.04988,2019-05-01 00:00:00,00:04:27.796666,2019-05-01 00:00:00,00:06:23.150000,36.17780,-86.78660,36.17740,-86.78760,"[[36.177699,-86.786477],[36.177711,-86.786469]...",2019-05-02 05:30:24.207000,2019-05-01 00:04:27.796666,2019-05-01 00:06:23.150000,0 days 00:01:55.353334


In [8]:
may_trip.sort_values('time_diff')

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
185052,2019-05-26 19:15:08.280000,Bird,BRD3308,PoweredUNKNOWN,0.0,0.00000,2019-05-26 00:00:00,19:17:03.860000,2019-05-26 00:00:00,19:17:04.733333,36.164700,-86.781000,36.164700,-86.781000,[],2019-05-27 05:31:57.320000,2019-05-26 19:17:03.860000,2019-05-26 19:17:04.733333,0 days 00:00:00.873333
160671,2019-05-24 23:03:08.200000,Bird,BRD3253,PoweredUNKNOWN,0.0,0.00000,2019-05-24 00:00:00,23:04:53.123333,2019-05-24 00:00:00,23:04:54.103333,36.161700,-86.774000,36.161700,-86.774000,[],2019-05-25 05:31:44.330000,2019-05-24 23:04:53.123333,2019-05-24 23:04:54.103333,0 days 00:00:00.980000
198810,2019-05-27 20:16:14.983000,Bird,BRD2603,PoweredUNKNOWN,0.0,0.00000,2019-05-27 00:00:00,20:16:36.093333,2019-05-27 00:00:00,20:16:37.203333,36.163700,-86.776300,36.163700,-86.776300,[],2019-05-28 05:31:55.520000,2019-05-27 20:16:36.093333,2019-05-27 20:16:37.203333,0 days 00:00:01.110000
155503,2019-05-24 12:52:40.937000,Bird,BRD91,PoweredUNKNOWN,0.0,0.00000,2019-05-24 00:00:00,12:54:37.006666,2019-05-24 00:00:00,12:54:38.163333,36.155400,-86.781100,36.155400,-86.781100,[],2019-05-25 05:30:57.263000,2019-05-24 12:54:37.006666,2019-05-24 12:54:38.163333,0 days 00:00:01.156667
161433,2019-05-25 00:18:11.863000,Bird,BRD3697,PoweredUNKNOWN,0.0,0.00000,2019-05-25 00:00:00,00:19:35.916666,2019-05-25 00:00:00,00:19:37.200000,36.173600,-86.785400,36.173600,-86.785400,[],2019-05-26 05:30:26.473000,2019-05-25 00:19:35.916666,2019-05-25 00:19:37.200000,0 days 00:00:01.283334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112598,2019-05-18 14:20:06.423000,SPIN,SPI43,Powered5839762,6496.0,193.56956,2019-05-14 00:00:00,01:59:57,2019-05-18 00:00:00,14:15:24,36.162934,-86.777197,36.163460,-86.777320,"[[36.16344,-86.77733],[36.16344,-86.77733],[36...",2019-05-19 09:40:48.260000,2019-05-14 01:59:57.000000,2019-05-18 14:15:24.000000,4 days 12:15:27
216554,2019-05-30 23:41:06.850000,SPIN,SPI1308,Powered1932731,7332.0,1706.03680,2019-05-25 00:00:00,21:25:54,2019-05-30 00:00:00,23:37:35,36.163341,-86.771313,36.164760,-86.776830,"[[36.1647761,-86.7767455],[36.16476,-86.77683]...",2019-05-31 08:54:59.280000,2019-05-25 21:25:54.000000,2019-05-30 23:37:35.000000,5 days 02:11:41
161280,2019-05-24 23:50:53.553000,SPIN,SPI450,Powered2043594,9000.0,7700.13148,2019-05-18 00:00:00,17:47:30,2019-05-24 00:00:00,23:47:10,36.163099,-86.776226,36.149981,-86.796673,"[[36.14998057022332,-86.796672726130851],[36.1...",2019-05-25 08:36:19.873000,2019-05-18 17:47:30.000000,2019-05-24 23:47:10.000000,6 days 05:59:40
168630,2019-05-25 18:16:36.020000,SPIN,SPI17,Powered6197635,11156.0,3517.06048,2019-05-18 00:00:00,00:18:57,2019-05-25 00:00:00,18:14:12,36.159250,-86.772751,36.163797,-86.783278,"[[36.163834653245111,-86.783087877090026],[36....",2019-05-26 09:10:01.160000,2019-05-18 00:18:57.000000,2019-05-25 18:14:12.000000,7 days 17:55:15


In [9]:
bar_may = may_trip[['companyName', 'time_diff']]
bar_may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224969 entries, 0 to 224968
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype          
---  ------       --------------   -----          
 0   companyName  224969 non-null  object         
 1   time_diff    224969 non-null  timedelta64[ns]
dtypes: object(1), timedelta64[ns](1)
memory usage: 3.4+ MB


In [10]:
may_trip.companyName.value_counts()

Lime             102569
Lyft              54271
Bird              53496
SPIN               8471
Bolt Mobility      4912
Gotcha              650
JUMP                600
Name: companyName, dtype: int64

In [32]:
lime = may_trip.loc[(may_trip['companyName'] == 'Lime')]
lyft = may_trip.loc[(may_trip['companyName'] == 'Lyft')]
bird = may_trip.loc[(may_trip['companyName'] == 'Bird')]
spin = may_trip.loc[(may_trip['companyName'] == 'SPIN')]
bolt = may_trip.loc[(may_trip['companyName'] == 'Bolt Mobility')]
gotcha = may_trip.loc[(may_trip['companyName'] == 'Gotcha')]
jump = may_trip.loc[(may_trip['companyName'] == 'JUMP')]

In [12]:
lime.shape

(102569, 19)

In [33]:
lyft.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
1,2019-05-01 00:03:33.147000,Lyft,LFT5,Powered296631,1.715600,1371.39112,2019-05-01 00:00:00,00:01:50.090000,2019-05-01 00:00:00,00:03:33.026666,36.15797,-86.77896,36.16054,-86.77689,"[[36.15797,-86.77896],[36.15795,-86.77873],[36...",2019-05-02 07:20:32.757000,2019-05-01 00:01:50.090000,2019-05-01 00:03:33.026666,0 days 00:01:42.936666
6,2019-05-01 00:07:55.880000,Lyft,LFT10,Powered329418,1.093917,0.00000,2019-05-01 00:00:00,00:06:50.106666,2019-05-01 00:00:00,00:07:55.740000,36.15168,-86.78408,36.15171,-86.78411,"[[36.15168,-86.78408],[36.15171,-86.78411]]",2019-05-02 07:20:32.890000,2019-05-01 00:06:50.106666,2019-05-01 00:07:55.740000,0 days 00:01:05.633334
7,2019-05-01 00:10:50.240000,Lyft,LFT13,Powered438572,2.954383,1161.41736,2019-05-01 00:00:00,00:07:52.853333,2019-05-01 00:00:00,00:10:50.116666,36.15193,-86.78439,36.15463,-86.78492,"[[36.15193,-86.78439],[36.15188,-86.78432],[36...",2019-05-02 07:20:32.990000,2019-05-01 00:07:52.853333,2019-05-01 00:10:50.116666,0 days 00:02:57.263333
15,2019-05-01 00:10:58.173000,Lyft,LFT14,Powered689177,2.780000,1161.41736,2019-05-01 00:00:00,00:08:11.206666,2019-05-01 00:00:00,00:10:58.006666,36.15186,-86.78441,36.15461,-86.78494,"[[36.15186,-86.78441],[36.15179,-86.78431],[36...",2019-05-02 07:20:33.020000,2019-05-01 00:08:11.206666,2019-05-01 00:10:58.006666,0 days 00:02:46.800000
16,2019-05-01 00:11:26.627000,Lyft,LFT16,Powered901629,6.436200,4908.13664,2019-05-01 00:00:00,00:05:00.326666,2019-05-01 00:00:00,00:11:26.500000,36.16211,-86.77528,36.15151,-86.77238,"[[36.16211,-86.77528],[36.16206,-86.7753],[36....",2019-05-02 07:20:33.087000,2019-05-01 00:05:00.326666,2019-05-01 00:11:26.500000,0 days 00:06:26.173334


In [55]:
jump.head(10)

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
100007,2019-05-17 10:31:57,JUMP,JMP425,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,5.735817,1161.6,2019-05-17 00:00:00,05:44:32,2019-05-17 00:00:00,05:50:16,36.167904,-86.782970,36.167953,-86.783110,"[[""36.16797"",""-86.782968""],[""36.167975"",""-86.7...",2019-05-18 06:20:19.380000,2019-05-17 05:44:32,2019-05-17 05:50:16,0 days 00:05:44
100008,2019-05-17 10:31:57,JUMP,JMP382,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,17.605667,6969.6,2019-05-17 00:00:00,04:33:39,2019-05-17 00:00:00,04:51:16,36.161507,-86.775980,36.168410,-86.767180,"[[""36.161466"",""-86.775936""],[""36.161466"",""-86....",2019-05-18 06:20:17.900000,2019-05-17 04:33:39,2019-05-17 04:51:16,0 days 00:17:37
100009,2019-05-17 10:31:57,JUMP,JMP391,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,3.145083,633.6,2019-05-17 00:00:00,05:03:35,2019-05-17 00:00:00,05:06:43,36.158195,-86.784030,36.158676,-86.782610,"[[""36.158383"",""-86.784003""],[""36.158096"",""-86....",2019-05-18 06:20:18.233000,2019-05-17 05:03:35,2019-05-17 05:06:43,0 days 00:03:08
100010,2019-05-17 10:31:57,JUMP,JMP461,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,2.921533,1108.8,2019-05-17 00:00:00,07:26:38,2019-05-17 00:00:00,07:29:34,36.163870,-86.774780,36.164394,-86.775670,"[[""36.16387"",""-86.774781""],[""36.16387"",""-86.77...",2019-05-18 06:20:35.663000,2019-05-17 07:26:38,2019-05-17 07:29:34,0 days 00:02:56
100011,2019-05-17 10:31:57,JUMP,JMP415,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,25.616600,17054.4,2019-05-17 00:00:00,05:11:18,2019-05-17 00:00:00,05:36:55,36.157550,-86.792755,36.182472,-86.806590,"[[""36.157531"",""-86.792738""],[""36.157443"",""-86....",2019-05-18 06:20:18.973000,2019-05-17 05:11:18,2019-05-17 05:36:55,0 days 00:25:37
100012,2019-05-17 10:31:57,JUMP,JMP452,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,11.141167,4488.0,2019-05-17 00:00:00,06:56:04,2019-05-17 00:00:00,07:07:13,36.176110,-86.792890,36.179626,-86.789230,"[[""36.17611"",""-86.792891""],[""36.17611"",""-86.79...",2019-05-18 06:20:20.213000,2019-05-17 06:56:04,2019-05-17 07:07:13,0 days 00:11:09
100013,2019-05-17 10:31:57,JUMP,JMP443,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,9.738183,7867.2,2019-05-17 00:00:00,06:29:08,2019-05-17 00:00:00,06:38:53,36.161190,-86.777405,36.179653,-86.790825,"[[""36.161165"",""-86.777511""],[""36.161175"",""-86....",2019-05-18 06:20:20.023000,2019-05-17 06:29:08,2019-05-17 06:38:53,0 days 00:09:45
100014,2019-05-17 10:31:57,JUMP,JMP384,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,25.911818,8184.0,2019-05-17 00:00:00,04:28:51,2019-05-17 00:00:00,04:54:46,36.137924,-86.805084,36.145424,-86.809710,"[[""36.137926"",""-86.805088""],[""36.137926"",""-86....",2019-05-18 06:20:18.040000,2019-05-17 04:28:51,2019-05-17 04:54:46,0 days 00:25:55
100015,2019-05-17 10:31:57,JUMP,JMP393,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,11.225650,7814.4,2019-05-17 00:00:00,04:56:27,2019-05-17 00:00:00,05:07:41,36.161230,-86.777534,36.145206,-86.784730,"[[""36.161365"",""-86.777686""],[""36.161186"",""-86....",2019-05-18 06:20:18.473000,2019-05-17 04:56:27,2019-05-17 05:07:41,0 days 00:11:14
100016,2019-05-17 10:31:57,JUMP,JMP379,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,5.310083,1531.2,2019-05-17 00:00:00,04:41:10,2019-05-17 00:00:00,04:46:29,36.155620,-86.780850,36.154870,-86.776750,"[[""36.155621"",""-86.780855""],[""36.155628"",""-86....",2019-05-18 06:20:17.763000,2019-05-17 04:41:10,2019-05-17 04:46:29,0 days 00:05:19


In [14]:
lime_time_1 = lime.time_diff/pd.Timedelta(hours=1)
lime_time_1.sum()

28032.319722222222

In [16]:
gotcha_time_1 = gotcha.time_diff/pd.Timedelta(hours=1)
gotcha_time_1.sum()

115.6425

In [62]:
lime_5 = lime.time_diff/pd.Timedelta(minutes=1)
round(lime_5.mean(),3)

16.398

In [56]:
lyft_5 = lyft.time_diff/pd.Timedelta(minutes=1)
round(lyft_5.mean(),3)

16.162

In [57]:
bird_5 = bird.time_diff/pd.Timedelta(minutes=1)
round(bird_5.mean(),3)

13.512

In [58]:
spin_5 = spin.time_diff/pd.Timedelta(minutes=1)
round(spin_5.mean(),3)

34.026

In [59]:
bolt_5 = bolt.time_diff/pd.Timedelta(minutes=1)
round(bolt_5.mean(),3)

20.363

In [60]:
gotcha_5 = gotcha.time_diff/pd.Timedelta(minutes=1)
round(gotcha_5.mean(),3)

10.675

In [61]:
jump_5 = jump.time_diff/pd.Timedelta(minutes=1)
round(jump_5.mean(),3)

43.586